이전 과정과 달리 max_depth으로 하면 성능이 달라지나요 ??
-> 아마 달라질 듯, 모든 값을 다 시도할 수 는 없지만 시간이 허락하는 대로 테스트하려 한다

이런저런 값으로 모델을 많이 만들어서 테스트 세트로 평가하면 결국 테스트 세트에 잘 맞는 모델이 만들어지는 것 아닌가? <br>
그러나 테스트 세트를 사용해 자꾸 성능을 확인하다 보면 점점 테스트 세트에 맞추게 된다

테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다 <br>
가능한 한 딱 한번만 사용하는 것이 좋다.

그렇다면 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝을 어떻게 할 수 있을까? <br>
게다가 결정 트리는 테스트해 볼 매개변수가 많다.

### 검증 세트

테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다. <br>
테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 **훈련 세트를 또 나누는 것**이다.

이 방법은 너무 단순해서 이상할 수도 있지만, 실제로 많이 사용하는 방법이다. <br>
전체 데이터 중 20%를 테스트 세트로 만들고 나머지 80%를 훈련 세트로 만들었다.

이 훈련 세트 중에서 다시 20%를 떼어 검증 세트로 만든다.

훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다. <br>
이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다.<br>


그 다음 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델 다시 훈련한다. <br>
마지막에 테스트 세트에서 최종 점수를 평가한다. 아마도 실전에 투입했을 때 테스트 점수와 비슷한 성능을 기대할 수 있을 것이다.

이전 절에서 사용했던 데이터를 다시 불러와 검증 세트를 만들어 보겠다. 먼저 판다스로 csv 데이터를 읽는다.

In [3]:
import pandas as pd 
wine = pd.read_csv("https://bit.ly/wine_csv_data")

ModuleNotFoundError: No module named 'pandas'

In [1]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

NameError: name 'wine' is not defined